In [2]:
import os
import pandas as pd
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer,BertModel
import logging
import difflib

In [4]:
def sim_Jaccard (str1,str2) :
    set1 = set( str1.lower().replace(';',' ').replace(',',' ').replace('.',' ').replace(':',' ').replace('&',' ').
               replace('/',' ').replace('\'',' ').replace('(author)',' ').replace('(joint author)',' ').split() )
    set2 = set( str2.lower().replace(';',' ').replace(',',' ').replace('.',' ').replace(':',' ').replace('&',' ').
               replace('/',' ').replace('\'',' ').replace('(author)',' ').replace('(joint author)',' ').split() )
    return len(set1&set2)/len(set1|set2)

In [45]:
#查验Jaccard标签效果，同时发现一些问题下存在全1或全0标签，后期可考虑删除等特殊处理
def watch_data (i):
    label_row = label.loc[i]
    print(label_row['author'])
    data_rows = data_golden[data_golden['isbn']==label_row['isbn']]
    for index,row in data_rows.iterrows():
        print(sim_Jaccard(label_row['author'],row['author']))
        
# 5025个1标签 -> 5619个1标签 说明某些字符的存在客观上拉低了标签的准确性
sum(data_golden['label'])

In [5]:
#读取并剪切新数据集

#读取文件夹中所有数据文件
path = './DataSet/vldbBook/book'
files = os.listdir(path)
data = pd.DataFrame()
for file in files:
    if 'book_detail' in file:
        data = data.append(pd.read_table(path+'/'+file,low_memory=False))
data.reset_index(inplace=True,drop=True)
data = data.drop(columns=['ISBN_13','location','seller_link','seller_date','publisher','publish_date','binding','book_condition',
                          'illustrator','dust_jacket_condition','signed','edition','price','link','used','small_cate'])
data.rename(columns={'ISBN_10':'isbn','title':'name','authors':'author','seller':'source'},inplace=True)
data.drop_duplicates(subset=['isbn','source'],inplace=True)
data['author'].fillna('Not Available', inplace=True)
data.reset_index(inplace=True,drop=True)
print("length of all data : ",len(data))
#data有2245349行

label = pd.read_table('./DataSet/vldbBook/book_truth.txt',low_memory=False,index_col=0)
label.rename(columns={'isbn_10':'isbn','authors_truth':'author'},inplace=True)
print("length of all label : ",len(label))
#label有407行，以isbn为index，以author为value

#提取data中有标签的数据，加上标签构成data_golden
data_golden = pd.DataFrame()
for index1,row1 in label.iterrows():
    str1 = row1['author']
    if(type(str1)==float):
        print(index1,row1)
        break
    data_slice = pd.DataFrame(data[data['isbn']==index1])
    data_slice['label'] = None
    for index2,row2 in data_slice.iterrows():
        str2 = row2['author']
        if(type(str2)==float):
            print(index2,row2)
            break
        data_slice.loc[index2,'label'] = ( sim_Jaccard(str1,str2)>=0.8 )   
        #这里修改成功了？离谱
        #if(sim_Jaccard(str1,str2)>=0.8):
        #    row2['label'] = True
        #else:
        #    row2['label'] = False
    data_golden = data_golden.append(data_slice)
data_golden.reset_index(inplace=True,drop=True)
print('length of data_golden : ',len(data_golden))
#data_golden去冗余后剩余11220行4列，来自1774 source、407 book，共有5609个1标签
print('num of 1 labels : ',sum(data_golden['label']))
seller_list = data_golden['source'].drop_duplicates().reset_index(drop=True)
book_list = data_golden['isbn'].drop_duplicates().reset_index(drop=True)
print("num of sellers : ",len(seller_list))
print("num of books : ",len(book_list))


#保存中间结果
data_golden.to_csv( './DataSet/vldbBook/data_golden/data_golden.txt' , sep='\t' , index=False )

#读取中间结果
#data_golden=pd.read_csv( './DataSet/vldbBook/data_golden2/data_golden.txt' , sep='\t' )

length of all data :  2245349
length of all label :  407
length of data_golden :  11220
num of 1 labels :  5609
num of sellers :  1774
num of books :  407


In [4]:
#加载bert模型和分词器
logging.basicConfig(level=logging.INFO)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\zxr\.pytorch_pretrained_bert\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\zxr\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file C:\Users\zxr\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir C:\Users\zxr\AppData\Local\Temp\tmp65rqas27
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_pr

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [5]:
#尝试使用bert分别提取各列语义特征作为GCN的输入feature
def feedColumnIntoBert(feature):
    for i in range(0,len(feature)):
        feature.loc[i] = '[CLS] '+str(feature.loc[i])+' [SEP]'
        feature.loc[i] = tokenizer.tokenize(feature.loc[i])
        feature.loc[i] = tokenizer.convert_tokens_to_ids(feature.loc[i])
        feature.loc[i] = torch.tensor([feature.loc[i]])
        with torch.no_grad():
            encoded_layers,_ = model(feature.loc[i])
        feature.loc[i] = encoded_layers[-1][0][0]+encoded_layers[-2][0][0]+encoded_layers[-3][0][0]+encoded_layers[-4][0][0]
    return feature
feature1 = feedColumnIntoBert(pd.Series(data_golden['source']).copy())
feature2 = feedColumnIntoBert(pd.Series(data_golden['name']).copy())
feature3 = feedColumnIntoBert(pd.Series(data_golden['author']).copy())
for i in range(0,len(feature1)):
    feature1.loc[i] = torch.cat((feature1.loc[i],feature2.loc[i],feature3.loc[i]))
#torch.save( feature1 , './DataSet/vldbBook/data_golden/bertEncodeFull.pt' )

In [8]:
#根据dataframe构建DGL图，添加如此多的边是否会起到恰当作用是我目前考虑的一个问题；
#传播会使得feature膨胀，是否需要重新设计GCN中的传播函数？

#1.添加一个连边限制，同source且同big_cate可连边，仅同source不可连边
#2.每一个节点上增加一个自环

import dgl
import dgl.function as fn
from dgl import DGLGraph

import pickle

def generate_DGLGraph(df):
    g = DGLGraph()
    g.add_nodes(df.shape[0])
    
    source_list = df['source'].drop_duplicates().reset_index(drop=True)
    isbn_list = df['isbn'].drop_duplicates().reset_index(drop=True)
    
    for index,row in df.iterrows():
        g.add_edge(index,index)
    
    for index,value in source_list.iteritems():
        df_slice = df[df['source']==value]
    #for k in range(0,1):
    #    df_slice = df[df['source']==source_list.loc[k]]
        for i in range(0,len(df_slice)):
            for j in range(i+1,len(df_slice)):
                if(df_slice.iloc[i]['big_cate']==df_slice.iloc[j]['big_cate']):
                    g.add_edge(df_slice.iloc[i].name,df_slice.iloc[j].name)
                    g.add_edge(df_slice.iloc[j].name,df_slice.iloc[i].name)

    for index,value in isbn_list.iteritems():
        df_slice = df[df['isbn']==value]
    #for k in range(0,1):
    #    df_slice = df[df['isbn']==isbn_list.loc[k]]
        for i in range(0,len(df_slice)):
            for j in range(i+1,len(df_slice)):
                if( sim_Jaccard( df_slice.iloc[i]['author'] , df_slice.iloc[j]['author'] ) >= 0.8 ):
                    g.add_edge(df_slice.iloc[i].name,df_slice.iloc[j].name)
                    g.add_edge(df_slice.iloc[j].name,df_slice.iloc[i].name)

    return g

#图中共有979230条有向边 -> 463000 -> 474220
g = generate_DGLGraph(data_golden)

#使用pickle将图存储在文件中以便复用
file = open('./DataSet/vldbBook/data_golden/graph.pickle', 'wb')
pickle.dump(g, file)
file.close()

In [10]:
g.edges()[0].shape

torch.Size([474220])